In [84]:
import requests
import pandas as pd
from sqlalchemy import create_engine
from pathlib import Path
from google.cloud import bigquery

<b> Fetch player data from Riot API <b>

In [51]:
regions = ["BR1", "VN2", "KR"]
queue = "RANKED_SOLO_5x5"
total_users = []
request_urls = []

api_key = "RGAPI-51d9ac2f-fbaa-4d6e-a378-b645b77d20fa"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
    "Accept-Language": "vi-VN,vi;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": api_key
}


In [52]:
for region in regions:
    challenger_url = 'https://{}.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/{}'.format(region,queue)
    grandmaster_url = 'https://{}.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/{}'.format(region,queue)
    #master_url = 'https://{}.api.riotgames.com/lol/league/v4/masterleagues/by-queue/{}'.format(region,queue)
    
    request_urls.append(challenger_url)
    request_urls.append(grandmaster_url)
    #request_urls.append(master_url)

In [53]:
for x in request_urls:
    temp_a = x.replace('//', '.')
    temp_b = temp_a.split(".")
    region = temp_b[1]
    response = requests.get(x, headers=headers)
        
    if response.status_code == 200:
        print(f"Region: {region}, Tier: {response.json()['tier']}, Queue: {response.json()['queue']}, Total Players: {len(response.json()['entries'])}")

        for y in response.json()['entries']:
            y['tier'] = response.json()['tier']
            y['request_region'] = region
            y['queue'] = queue
            total_users.append(y)
    else:
        print(f"'Request error, response code: {response.status_code}")

Region: BR1, Tier: CHALLENGER, Queue: RANKED_SOLO_5x5, Total Players: 200
Region: BR1, Tier: GRANDMASTER, Queue: RANKED_SOLO_5x5, Total Players: 500
Region: VN2, Tier: CHALLENGER, Queue: RANKED_SOLO_5x5, Total Players: 300
Region: VN2, Tier: GRANDMASTER, Queue: RANKED_SOLO_5x5, Total Players: 700
Region: KR, Tier: CHALLENGER, Queue: RANKED_SOLO_5x5, Total Players: 300
Region: KR, Tier: GRANDMASTER, Queue: RANKED_SOLO_5x5, Total Players: 700


In [54]:
print(len(total_users))

2700


In [56]:
total_users[0]

{'summonerId': 'vbPSm9gW0VTclg9zPLWqNGHilUfa9c8Ogu45mSEKIMWy7oo',
 'summonerName': 'Fuuu',
 'leaguePoints': 1147,
 'rank': 'I',
 'wins': 170,
 'losses': 133,
 'veteran': False,
 'inactive': False,
 'freshBlood': False,
 'hotStreak': False,
 'tier': 'CHALLENGER',
 'request_region': 'BR1',
 'queue': 'RANKED_SOLO_5x5'}

<b> Convert data to DataFrame <b>

In [85]:
df = pd.DataFrame(total_users)
df.head()

,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak,tier,request_region,queue
0,vbPSm9gW0VTclg9zPLWqNGHilUfa9c8Ogu45mSEKIMWy7oo,Fuuu,1147,I,170,133,False,False,False,False,CHALLENGER,BR1,RANKED_SOLO_5x5
1,URWMt8chEwazuh6xIkltkGK7Qt4QkZ5dY3iL_O3l3zzxOg,twitch nicklink,1990,I,307,215,True,False,False,False,CHALLENGER,BR1,RANKED_SOLO_5x5
2,F2h3GIj9_N8bo7sCyo7oA90c0GRoeHnLz14L-Fq2G7ew8g,angustia33,1105,I,112,77,False,False,False,False,CHALLENGER,BR1,RANKED_SOLO_5x5
3,jYvbPWSRoOzD-yyUEKHYBLah3i7Hx7iQEZB-aS7PhxddtC8,PontoDoiss,1048,I,207,170,False,False,False,True,CHALLENGER,BR1,RANKED_SOLO_5x5
4,DrrPUZThWxVNMprgnzAy2DQhHKAR50-kgU1XLCNN3uYMxK0,kimoyi,1129,I,80,41,False,False,True,False,CHALLENGER,BR1,RANKED_SOLO_5x5


In [58]:
df.shape

(2700, 13)

In [86]:
df2 = df.dropna()

In [60]:
df2.shape

(2700, 13)

<b> Loading data to MySQL <b>

In [61]:
engine = create_engine('mysql+mysqlconnector://root:@localhost:3306/world')
engine.connect()

In [63]:
# create table in schema
print(pd.io.sql.get_schema(df2, name='league-v4-2.0', con=engine))


CREATE TABLE `league-v4-2.0` (
	`summonerId` TEXT, 
	`summonerName` TEXT, 
	`leaguePoints` BIGINT, 
	`rank` TEXT, 
	wins BIGINT, 
	losses BIGINT, 
	veteran BOOL, 
	inactive BOOL, 
	`freshBlood` BOOL, 
	`hotStreak` BOOL, 
	tier TEXT, 
	request_region TEXT, 
	queue TEXT
)




In [64]:
%time df2.to_sql(name='league-v4-2.0', con=engine, if_exists='replace')

CPU times: total: 109 ms
Wall time: 398 ms


-1

In [93]:
filepath = Path('dataset/out.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df2.to_csv(filepath)  

In [87]:
# Construct a BigQuery client object.
client = bigquery.Client()

In [88]:
job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("summonerID", "STRING"),
        bigquery.SchemaField("summonerName", "STRING"),
        bigquery.SchemaField("leaguePoints", "INTEGER"),
        bigquery.SchemaField("rank", "STRING"),
        bigquery.SchemaField("wins", "INTEGER"),
        bigquery.SchemaField("losses", "INTEGER"),
        bigquery.SchemaField("veteran", "BOOLEAN"),
        bigquery.SchemaField("inactive", "BOOLEAN"),
        bigquery.SchemaField("freshBlood", "BOOLEAN"),
        bigquery.SchemaField("hotStreak", "BOOLEAN"),
        bigquery.SchemaField("tier", "STRING"),
        bigquery.SchemaField("request_region", "STRING"),
        bigquery.SchemaField("queue", "STRING")
    ], 
    write_disposition="WRITE_TRUNCATE",
)
#WRITE_TRUNCATE write disposition it replaces the table with the loaded data.

In [92]:
table_id = "northern-union-373114.trips_data_all.league-v4-2.0"
job = client.load_table_from_dataframe(
    df2, table_id, job_config=job_config
)  # Make an API request.
job.result()  # Wait for the job to complete.